## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

from utils import get_calibration_points, undistorter_from_pickle
%matplotlib qt

objpoints, imgpoints = get_calibration_points()

##
# Apply the objpoints and imgpoints to get the calibration results
# Use the calibration results to undistort a test image. Visualize the result
##
#%%
import pickle
%matplotlib inline

# Test undistortion on an image
img = cv2.imread('./camera_cal/calibration1.jpg')
img_size = (img.shape[1], img.shape[0])

# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
    objpoints,
    imgpoints,
    img_size,
    None,
    None)

# Save the camera calibration result for later use (we won't worry about
# rvecs / tvecs)
pickle_path = "./wide_dist_pickle.p"
dist_pickle = {}
dist_pickle["dist"] = dist
dist_pickle["mtx"] = mtx
pickle.dump(dist_pickle, open(pickle_path, "wb"))

In [2]:
undistort_img = undistorter_from_pickle(pickle_path)

In [3]:
from utils import pipeline

In [4]:
from utils import get_warper_from_image_shape
test_image = cv2.imread('./test_images/straight_lines1.jpg')
warper = get_warper_from_image_shape(test_image.shape)

In [5]:
from line import Line
from line import LaneLines
from utils import sliding_windows, next_frame, get_drawable_lanes, visualize_lines_on_image, get_curvature
import math

lane_lines = LaneLines()

def process_image(image):
    
    undistorted_image = undistort_img(image)
    pipelined_image = pipeline(undistorted_image)
    warped = warper(pipelined_image)
    
    lane_lines.process_warped_image(warped)
    
    left_fitx, right_fitx, ploty = get_drawable_lanes(
        lane_lines.left_line.current_fit, 
        lane_lines.right_line.current_fit, 
        warped.shape
    )

    return visualize_lines_on_image(ploty, left_fitx, right_fitx, warped)
    
    

In [6]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [7]:
video_output = 'lane_found_output.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video lane_found_output.mp4
[MoviePy] Writing video lane_found_output.mp4


100%|█████████▉| 1260/1261 [03:05<00:00,  6.51it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: lane_found_output.mp4 

CPU times: user 2min 49s, sys: 52.3 s, total: 3min 41s
Wall time: 3min 5s


In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))